In [7]:
%load_ext autoreload
%autoreload 2

import json
import pickle
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from multiprocessing.pool import ThreadPool
from multiprocessing import Process, Manager, Pool
from collections import Counter
import sys

sys.path.append("../")
from utils.parse_arxiv import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
chunks = pd.read_json(
    "../data/arxiv/arxiv-metadata-oai-snapshot.json", lines=True, chunksize=50000
)

In [3]:
chunks.chunksize

50000

In [4]:
# manager = Manager()

# author_dict = manager.dict()
# cat_dict = manager.dict()
# year_dict = manager.dict()
author_dict = {}
cat_dict = {}
journal_dict = {}
year_dict = {}

counter = 0
for chunk in tqdm(chunks):
    df = chunk
    (
        intermediate_authors_stats,
        intermediate_cat_stats,
        intermediate_jou_stats,
        intermediate_year_stats,
    ) = gather_stats(df)
    author_dict = merge_add_dict(author_dict, intermediate_authors_stats)
    cat_dict = merge_add_dict(cat_dict, intermediate_cat_stats)
    journal_dict = merge_add_dict(journal_dict, intermediate_jou_stats)
    year_dict = merge_add_dict(year_dict, intermediate_year_stats)

    counter += len(df)
    # print("processed:", counter)
    # print("aut stats => ", len(author_dict))
    # print("cat stats => ", len(cat_dict))
    # print("journal stats => ", len(journal_dict))
    # print("year stats => ", len(year_dict))
    # break

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [7]:
author_dict = sort_dict(author_dict)
cat_dict = sort_dict(cat_dict)
journal_dict = sort_dict(journal_dict)
year_dict = sort_dict(year_dict)

In [8]:
# with open('author.pickle', 'wb') as handle:
#     pickle.dump(author_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('cat.pickle', 'wb') as handle:
#     pickle.dump(cat_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('journal.pickle', 'wb') as handle:
#     pickle.dump(journal_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('date.pickle', 'wb') as handle:
#     pickle.dump(year_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
def parse_paper_id(paper_id):
    # making sure that the paper_id is a string!!!
    return "id_" + str(paper_id)

## Filter data for all metadata frequencies > 1

In [10]:
chunks = pd.read_json(
    "../data/arxiv/arxiv-metadata-oai-snapshot.json", lines=True, chunksize=50000
)
filtered_df = None
for i, chunk in enumerate(chunks):
    df = chunk
    df["cat_freq"] = df["categories"].map(cat_dict)
    df["journal_freq"] = df["journal-ref"].map(journal_dict)
    df["date_freq"] = df["update_date"].map(year_dict)
    df = df.loc[df["cat_freq"] > 1]
    print(df.shape)
    df = df.loc[df["journal_freq"] > 1]
    print(df.shape)
    df = df.loc[df["date_freq"] > 1]
    print(df.shape, "\n")
    if i == 0:
        filtered_df = df
    else:
        filtered_df = pd.concat([filtered_df, df])

(48969, 17)
(1, 17)
(1, 17) 

(48924, 17)
(0, 17)
(0, 17) 

(48846, 17)
(0, 17)
(0, 17) 

(48519, 17)
(3, 17)
(3, 17) 

(48301, 17)
(6, 17)
(6, 17) 

(48773, 17)
(80, 17)
(80, 17) 

(48871, 17)
(119, 17)
(119, 17) 

(48824, 17)
(80, 17)
(80, 17) 

(48999, 17)
(117, 17)
(117, 17) 

(49046, 17)
(94, 17)
(94, 17) 

(49055, 17)
(67, 17)
(67, 17) 

(49000, 17)
(54, 17)
(54, 17) 

(48987, 17)
(62, 17)
(62, 17) 

(49023, 17)
(80, 17)
(80, 17) 

(49039, 17)
(122, 17)
(122, 17) 

(49049, 17)
(165, 17)
(165, 17) 

(48970, 17)
(201, 17)
(201, 17) 

(48956, 17)
(195, 17)
(195, 17) 

(48865, 17)
(316, 17)
(316, 17) 

(48945, 17)
(370, 17)
(370, 17) 

(49005, 17)
(406, 17)
(406, 17) 

(49018, 17)
(500, 17)
(499, 17) 

(48813, 17)
(575, 17)
(575, 17) 

(48605, 17)
(526, 17)
(526, 17) 

(48724, 17)
(579, 17)
(579, 17) 

(48602, 17)
(538, 17)
(538, 17) 

(48677, 17)
(554, 17)
(554, 17) 

(48740, 17)
(577, 17)
(577, 17) 

(48679, 17)
(563, 17)
(563, 17) 

(48697, 17)
(553, 17)
(553, 17) 

(48681, 17)
(5

In [10]:
# file = open("../data/arxiv/filtered_data.pickle", "rb")
# filtered_df = pickle.load(file)
# file.close()

from utils.parse_arxiv import *

In [20]:
filtered_df['title'] = filtered_df['title'].map(parse_title).values
filtered_df['abstract'] = filtered_df['abstract'].map(parse_title).values

In [22]:
def str_length(s):
    return len(s)

filtered_df['title_len'] = filtered_df['title'].map(str_length).values
filtered_df['abs_len'] = filtered_df['abstract'].map(str_length).values

In [37]:
filtered_df = filtered_df[filtered_df['title_len']>5]
# filtered_df[filtered_df['abs_len']<10]

print(len(filtered_df))
filtered_df.drop_duplicates(subset=['id'], inplace=True)
filtered_df.drop_duplicates(subset=['title'], inplace=True)
filtered_df.drop_duplicates(subset=['abstract'], inplace=True)
print(len(filtered_df))

11565
11540


In [23]:
filtered_df.head(3)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,cat_freq,journal_freq,date_freq,title_len,abs_len
40439,id_712.2518,Richard A. Klemm,Richard A. Klemm and Dmitri V. Efremov,Single-ion and exchange anisotropy effects and...,"30 pages, 14 figures, submitted to Phys. Rev. B","Phys. Rev. B 77, 184410 (2008).",10.1103/PhysRevB.77.184410,None,cond-mat.str-el cond-mat.mes-hall,None,We study single-ion and exchange anisotropy ef...,"[{'version': 'v1', 'created': 'Sat, 15 Dec 200...",2009-11-13,"[[Klemm, Richard A., ], [Efremov, Dmitri V., ]]",2081.0,2.0,5444.0,118,1625
151192,id_910.3128,Luis Lomel\'i,Guy Henniart and Luis Lomel\'i,Characterization of \gamma-factors: the Asai case,None,International Mathematics Research Notices 2012,10.1093/imrn/rns171,None,math.NT,http://arxiv.org/licenses/nonexclusive-distrib...,Let $E$ be a separable quadratic extension of ...,"[{'version': 'v1', 'created': 'Fri, 16 Oct 200...",2012-09-05,"[[Henniart, Guy, ], [Lomelí, Luis, ]]",18860.0,2.0,162.0,49,808
155604,id_911.1583,Johan Bollen,"Johan Bollen, Alberto Pepe and Huina Mao",Modeling public mood and emotion: Twitter sent...,submitted to WWW2010,Proceedings of the Fifth International AAAI Co...,None,None,cs.CY,http://arxiv.org/licenses/nonexclusive-distrib...,Microblogging is a form of online communicatio...,"[{'version': 'v1', 'created': 'Mon, 9 Nov 2009...",2011-03-29,"[[Bollen, Johan, ], [Pepe, Alberto, ], [Mao, H...",4379.0,2.0,126.0,80,1540


In [13]:
filtered_df['id'] = filtered_df['id'].apply(parse_paper_id)

len(filtered_df)

11566

In [14]:
filtered_df.head(3)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,cat_freq,journal_freq,date_freq
40439,id_712.2518,Richard A. Klemm,Richard A. Klemm and Dmitri V. Efremov,Single-ion and exchange anisotropy effects and...,"30 pages, 14 figures, submitted to Phys. Rev. B","Phys. Rev. B 77, 184410 (2008).",10.1103/PhysRevB.77.184410,None,cond-mat.str-el cond-mat.mes-hall,None,We study single-ion and exchange anisotropy ...,"[{'version': 'v1', 'created': 'Sat, 15 Dec 200...",2009-11-13,"[[Klemm, Richard A., ], [Efremov, Dmitri V., ]]",2081.0,2.0,5444.0
151192,id_910.3128,Luis Lomel\'i,Guy Henniart and Luis Lomel\'i,Characterization of \gamma-factors: the Asai case,None,International Mathematics Research Notices 2012,10.1093/imrn/rns171,None,math.NT,http://arxiv.org/licenses/nonexclusive-distrib...,Let $E$ be a separable quadratic extension o...,"[{'version': 'v1', 'created': 'Fri, 16 Oct 200...",2012-09-05,"[[Henniart, Guy, ], [Lomelí, Luis, ]]",18860.0,2.0,162.0
155604,id_911.1583,Johan Bollen,"Johan Bollen, Alberto Pepe and Huina Mao",Modeling public mood and emotion: Twitter sent...,submitted to WWW2010,Proceedings of the Fifth International AAAI Co...,None,None,cs.CY,http://arxiv.org/licenses/nonexclusive-distrib...,Microblogging is a form of online communicat...,"[{'version': 'v1', 'created': 'Mon, 9 Nov 2009...",2011-03-29,"[[Bollen, Johan, ], [Pepe, Alberto, ], [Mao, H...",4379.0,2.0,126.0


In [38]:
filtered_df.to_pickle("../data/arxiv/filtered_data.pickle")